In [1]:
## This is pretty much just a set up for MLFLOW (Going to learn this)
## I know the model is bad

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("../data/detroit_open_data_portal_property_sales.csv")

df = df[df['Property Class Code'] == 201]
df = df[df['Sale Price'] != 0]

df["Sale Date"] = pd.to_datetime(df["Sale Date"])
# Extract month and year
df["month"] = df["Sale Date"].dt.month
df["year"] = df["Sale Date"].dt.year

print(df.shape)
df.head(3)

c:\users\matt\desktop\dataprojects\detroit_property_prices\venv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (19,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(16030, 32)


,Sales ID,Parcel ID,Address,Sale Date,Sale Price,Grantor,Grantee,Liber Page,Terms of Sale,Sale Verification,Sale Instrument,sale_number,Property Transfer Percentage,Multi Parcel Sale,Property Class Code,Property Class Description,ECF Neighborhood,Neighborhood,Council District,Zip Code,Street Number,Street Prefix,Street Name,Street Type,Unit Number,Longitude,Latitude,ObjectId,x,y,month,year
199,1487769,01000236.,139 CADILLAC SQ,2012-07-30,1146000,139 CADILLAC SQUARE LLC,GNT HOLDINGS LLC,NaN,21-NOT USED/OTHER,PROPERTY TRANSFER AFFIDAVIT,PTA,1,100,False,201.0,COMMERCIAL,CCAPT,Downtown,6.0,48226,139.0,NaN,CADILLAC SQ,NaN,NaN,-83.043674,42.331956,2462,-9.244380e+06,5.210835e+06,7,2012
269,1487798,01000272-4,163 MADISON,2012-02-15,3000000,WAYCOR DEVELOPEMENT CO,GRAND VALLEY STATE UNIVERSITY,NaN,21-NOT USED/OTHER,PROPERTY TRANSFER AFFIDAVIT,PTA,1,100,False,201.0,COMMERCIAL,CCOB,Downtown,6.0,48226,163.0,NaN,MADISON,NaN,NaN,-83.048107,42.337446,2617,-9.244873e+06,5.211662e+06,2,2012
318,1488324,01000806.001,203 ERSKINE,2012-11-16,10100,WAYNE COUNTY TREASURER,ALMASS DOWNTOWN REAL ESTATE LLC,NaN,13-GOVERNMENT,TITLE COMPANY,PTA,1,100,False,201.0,COMMERCIAL,CCAPT,Brush Park,5.0,48201,203.0,NaN,ERSKINE,NaN,NaN,-83.054575,42.346710,3204,-9.245593e+06,5.213057e+06,11,2012


In [3]:
df['Neighborhood'].value_counts()

Warrendale              343
Fitzgerald/Marygrove    312
Outer Drive-Hayes       295
Barton-McFarland        289
Midtown                 282
                       ... 
Waterworks Park           2
Arden Park                2
Melvern Hill              1
Garden View               1
Boston Edison             1
Name: Neighborhood, Length: 199, dtype: int64

In [4]:
# Column Cleaning when needed

# Zip Code column remove whats after xxxxx-xxxx (remove this part: -xxxx)
df['Zip Code'] = df['Zip Code'].astype(str).str.split('-').str[0]
df['Zip Code'] = pd.to_numeric(df['Zip Code'], errors='coerce')

# Labeling Encoding (I Know this is wrong but its easier to just grab this column)
le = LabelEncoder()
df['Neighborhood_Encoded'] = le.fit_transform(df['Neighborhood'])

le2 = LabelEncoder()
df['Grantor_Encoded'] = le2.fit_transform(df['Grantor'])

le3 = LabelEncoder()
df['Street_Name_Encoded'] = le3.fit_transform(df['Street Name'])

In [5]:
def fit_model(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print(f"R-squared: {r2_score(y_test, y_pred)}")
    print(f"MSE: {mean_squared_error(y_test, y_pred)}")

    # Inspect the learned formula: y = mx + b
    print(f"Coefficient (slope): {model.coef_}")
    print(f"Intercept: {model.intercept_}")

In [6]:
# Initial Model
X = df[['month']]
y = df[['Sale Price']]

fit_model(X,y)

R-squared: 0.00014894967427581118
MSE: 164108294430751.4
Coefficient (slope): [[-38830.75964523]]
Intercept: [982660.49444753]


In [7]:
X = df[['month', 'year']]
y = df[['Sale Price']]

X = X.fillna(0)

fit_model(X,y)

R-squared: 0.0005857656339647299
MSE: 164036598629560.5
Coefficient (slope): [[-30488.63334885  74837.78718434]]
Intercept: [-1.50047675e+08]


In [8]:
X = df[['month', 'year', 'Latitude', 'Longitude']]
y = df[['Sale Price']]

X = X.fillna(0)

fit_model(X,y)

R-squared: 0.0010289040538333794
MSE: 163963865105644.4
Coefficient (slope): [[  -30523.1282765     78748.95410601 -2254266.9417639  -1140683.83085952]]
Intercept: [-1.57205806e+08]


In [9]:
X = df[['month', 'year', 'Latitude', 'Longitude', 'Zip Code']]
y = df[['Sale Price']]

X = X.fillna(0)

fit_model(X,y)

R-squared: 0.0012681595677175306
MSE: 163924595442124.16
Coefficient (slope): [[-3.11649894e+04  7.52018859e+04 -2.29577365e+06 -1.15254100e+06
   3.08631843e+01]]
Intercept: [-1.5075731e+08]


In [10]:
X = df[['month', 'year', 'Latitude', 'Longitude', 'Zip Code', 'Neighborhood_Encoded']]
y = df[['Sale Price']]

X = X.fillna(0)

fit_model(X,y)

R-squared: 0.0012242391216649162
MSE: 163931804225362.4
Coefficient (slope): [[-3.13144159e+04  7.45579155e+04 -2.27234344e+06 -1.13861332e+06
   3.02726749e+01 -1.75896074e+03]]
Intercept: [-1.49095387e+08]


In [11]:
X = df[['month', 'year', 'Latitude', 'Longitude', 'Zip Code', 'Neighborhood_Encoded',
       'Grantor_Encoded']]
y = df[['Sale Price']]

X = X.fillna(0)

fit_model(X,y)

R-squared: 0.0013071823972536922
MSE: 163918190518113.75
Coefficient (slope): [[-2.42745070e+04  7.13883964e+04 -2.19731409e+06 -1.10081715e+06
   2.93916211e+01 -1.70631701e+03 -5.35648072e+01]]
Intercept: [-1.42445651e+08]


In [12]:
X = df[['month', 'year', 'Latitude', 'Longitude', 'Zip Code', 'Neighborhood_Encoded',
       'Grantor_Encoded', 'Street_Name_Encoded']]
y = df[['Sale Price']]

X = X.fillna(0)

fit_model(X,y)

R-squared: 0.001278979314143447
MSE: 163922819567476.06
Coefficient (slope): [[-2.42304364e+04  7.14088147e+04 -2.17198953e+06 -1.08775667e+06
   3.11662507e+01 -1.78439920e+03 -5.38960518e+01  2.79147603e+02]]
Intercept: [-1.42670046e+08]


In [13]:
df.columns

Index(['Sales ID', 'Parcel ID', 'Address', 'Sale Date', 'Sale Price',
       'Grantor', 'Grantee', 'Liber Page', 'Terms of Sale',
       'Sale Verification', 'Sale Instrument', 'sale_number',
       'Property Transfer Percentage', 'Multi Parcel Sale',
       'Property Class Code', 'Property Class Description', 'ECF Neighborhood',
       'Neighborhood', 'Council District', 'Zip Code', 'Street Number',
       'Street Prefix', 'Street Name', 'Street Type', 'Unit Number',
       'Longitude', 'Latitude', 'ObjectId', 'x', 'y', 'month', 'year',
       'Neighborhood_Encoded', 'Grantor_Encoded', 'Street_Name_Encoded'],
      dtype='object')